# BN: approximate inverse of the left-hand side operator of the velocity system

Objective: estimate the condition number of the left-hand side operator of the pressure Poisson system for different truncated Taylor series of the inverse operator for the velocity system.

In [1]:
from scipy.sparse import identity
from scipy.sparse.linalg import norm

import pyibm

In [2]:
%matplotlib inline

In [3]:
pyibm.__version__

'0.1'

## Create stretched Cartesian grids

In [4]:
# Set parameters of the computational grid.
xlim, ylim = (-15.0, 15.0), (-15.0, 15.0)  # domain limits
xbox, ybox = (-0.75, 0.75), (-0.75, 0.75)  # uniform region
dx, dy = 0.05, 0.05  # grid spacings in the uniform region

config_x = [dict(start=xlim[0], end=xbox[0],
                 width=dx, stretching=1.03, reverse=True),
            dict(start=xbox[0], end=xbox[1], width=dx),
            dict(start=xbox[1], end=xlim[1],
                 width=dx, stretching=1.01)]

config_y = [dict(start=ylim[0], end=ybox[0],
                 width=dy, stretching=1.04, reverse=True),
            dict(start=ybox[0], end=ybox[1], width=dy),
            dict(start=ybox[1], end=ylim[1],
                 width=dy, stretching=1.04)]

config = {'x': dict(start=xlim[0], segments=config_x),
          'y': dict(start=ylim[0], segments=config_y)}

# Create staggered grids.
grid = pyibm.GridBase(config=config)  # vertex grid
gridc = pyibm.GridCellCentered(grid=grid)  # cell-centered grid
gridx = pyibm.GridFaceX(grid=grid)  # x-face centered grid
gridy = pyibm.GridFaceY(grid=grid)  # y-face centered grid
print(gridc)
print(gridx)
print(gridy)

Grid(size=38078, shape=(158, 241), gridlines=[
Gridline(start=-15.0, end=15.0, size=241),
Gridline(start=-15.0, end=15.0, size=158)])
Grid(size=37920, shape=(158, 240), gridlines=[
Gridline(start=-15.0, end=15.0, size=240),
Gridline(start=-15.0, end=15.0, size=158)])
Grid(size=37837, shape=(157, 241), gridlines=[
Gridline(start=-15.0, end=15.0, size=241),
Gridline(start=-15.0, end=15.0, size=157)])


## Assemble operators

In [5]:
# Assemble gradient operator.
GHat = pyibm.assemble_GHat(gridc, gridx, gridy)
pyibm.print_matrix_info(GHat, name='GHat')

# Assemble divergence operator.
DHat = pyibm.assemble_DHat(gridc, gridx, gridy)
pyibm.print_matrix_info(DHat, name='DHat')

Name:  GHat
Type:  <class 'scipy.sparse.csr.csr_matrix'>
Shape:  (75757, 38078)
Size:  151514
Min/Max:  -20.00000000000003 20.00000000000003
Name:  DHat
Type:  <class 'scipy.sparse.csr.csr_matrix'>
Shape:  (38078, 75757)
Size:  151514
Min/Max:  -0.5965515242499322 0.5965515242499322


In [6]:
# Assemble Laplacian operator.
LHat = pyibm.assemble_LHat(gridx, gridy)
pyibm.print_matrix_info(LHat, name='LHat')

Name:  LHat
Type:  <class 'scipy.sparse.csr.csr_matrix'>
Shape:  (75757, 75757)
Size:  377193
Symmetric:  False
Min/Max:  -1600.0000000000057 400.00000000000193


In [7]:
# Assemble the left-hand side operator of the velocity system.
# A = (1 / dt) I - (a_I / Re) L
dt = 0.01  # time-step size
Re = 100.0  # Reynolds number
a_I = 0.5  # implicit coefficient from Crank-Nicolson scheme
size = gridx.size + gridy.size
I = identity(size)
A = 1 / dt * I - a_I / Re * LHat
pyibm.print_matrix_info(A, name='A')

Name:  A
Type:  <class 'scipy.sparse.csr.csr_matrix'>
Shape:  (75757, 75757)
Size:  377193
Symmetric:  False
Min/Max:  -2.0000000000000098 108.00000000000003


In [8]:
# Assemble the approximate inverse of the implicit velocity operator A.
N = 3  # truncation order of the Taylor series expansion
BN = pyibm.assemble_BN(gridx, gridy, dt=dt,
                       alpha=a_I / Re, N=N, L=LHat)
pyibm.print_matrix_info(BN, name='BN')

Name:  BN
Type:  <class 'scipy.sparse.csr.csr_matrix'>
Shape:  (75757, 75757)
Size:  976889
Symmetric:  False
Min/Max:  2.2196330537567296e-10 0.009992380485539316


In [9]:
# Assemble the left-hand side operator of the Poisson system.
DBNG = DHat @ BN @ GHat
pyibm.print_matrix_info(DBNG, name='DBNG')

Name:  DBNG
Type:  <class 'scipy.sparse.csr.csr_matrix'>
Shape:  (38078, 38078)
Size:  940798
Symmetric:  True
Min/Max:  -0.22602590280207158 0.11045928068927477


## Condition number of Poisson operator

In [10]:
# Assemble the Poisson operator for different BN.
# Print information about the Poisson operator.
for N in [1, 2, 3]:
    print('\nTruncation order: ', N)
    # Assemble approximate inverse of A.
    BN = pyibm.assemble_BN(gridx, gridy, dt=dt,
                           alpha=a_I / Re, N=N, L=LHat)
    # Assemble Poisson operator.
    DBNG = DHat @ BN @ GHat
    pyibm.print_matrix_info(DBNG, name=f'DB{N}G')
    print('Condition number: ', pyibm.condition_number(DBNG))
    M = A @ BN - I
    print('L2(A BN - I) = ', norm(M))


Truncation order:  1
Name:  DB1G
Type:  <class 'scipy.sparse.csr.csr_matrix'>
Shape:  (38078, 38078)
Size:  189592
Symmetric:  True
Min/Max:  -0.23947519459473085 0.11931030484998661
Condition number:  (-6.462275462665142e-20, -0.47725770065559414, 7.385288717772574e+18)
L2(A BN - I) =  11.45883544217135

Truncation order:  2
Name:  DB2G
Type:  <class 'scipy.sparse.csr.csr_matrix'>
Shape:  (38078, 38078)
Size:  491028
Symmetric:  True
Min/Max:  -0.22506254728845054 0.10974139498652602
Condition number:  (-1.4048216940815085e-18, -0.4390159110136853, 3.1250650019376294e+17)
L2(A BN - I) =  0.9427787475515725

Truncation order:  3
Name:  DB3G
Type:  <class 'scipy.sparse.csr.csr_matrix'>
Shape:  (38078, 38078)
Size:  940798
Symmetric:  True
Min/Max:  -0.22602590280207158 0.11045928068927477
Condition number:  (-1.6106418647693223e-17, -0.4420800735789753, 2.744744708609014e+16)
L2(A BN - I) =  0.0978171975471883
